In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\amitt\\Documents\\ML-OPS\\text-summarization'

In [4]:
%pwd

'c:\\Users\\amitt\\Documents\\ML-OPS\\text-summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# types of data to be ingested
@dataclass
class DataIngestionConfig:
    root_dir : Path
    source_url : Path
    local_data_file: Path
    unzip_dir: Path

In [6]:
import src.textsummarizer.constant
print(dir(src.textsummarizer.constant))

['CONFIG_PATH', 'PARAMS_PATH', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'add']


In [7]:
from src.textsummarizer.constant import *
from src.textsummarizer.utils.common import read_yaml, create_directories

#### Basic Configuration steps

In [14]:
#  this goes into the configuration file under config folder
class ConfigurationManager:
    def __init__( self,config_filepath = CONFIG_PATH, params_filepath = PARAMS_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:

            config = self.config.data_ingestion

            # creating root directory for config.root_dir

            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(

                root_dir = config.root_dir,
                source_url = config.source_url,
                local_data_file = config.local_data_file,
                unzip_dir = config.unzip_dir

            )


            return data_ingestion_config


In [9]:
import os
import urllib.request as request
import zipfile
from src.textsummarizer.logging import logger
from src.textsummarizer.utils.common import get_size

#### Componenets

In [20]:
# this goes under components folder
class DataIngestion:

    def __init__(self,config:DataIngestionConfig):

        self.config = config

    def download_file(self):

        # if path does not exisit, retrieve the filename
        if not os.path.exists(self.config.local_data_file):

            filename, headers = request.urlretrieve(

            url = self.config.source_url,   
            filename = self.config.local_data_file

            )

            logger.info(f"{filename} download! with following info: \n{headers}")
        else:

            logger.info(f"file already exists {Path(self.config.local_data_file)}")

    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir

        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:

            zip_ref.extractall(unzip_path)



In [21]:
config = ConfigurationManager()

data_ingestion_config = config.get_data_ingestion_config()

[2025-02-14 16:39:41,050: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-14 16:39:41,052: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-14 16:39:41,053: INFO: common: created directory at: artifacts]
[2025-02-14 16:39:41,054: INFO: common: created directory at: artifacts/data_ingestion]


In [ ]:
data_ingestion = DataIngestion(config=data_ingestion_config)

data_ingestion.download_file()
data_ingestion.extract_zip_file()